# F1 Qualifying Position vs. Race Finish Position Performance Analysis (2021 - 2024 Season)

# Import necessary libraries

In [5]:
import pandas as pd
import numpy as np
import sqlite3
import os
import glob
import requests
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [ ]:
csv_folder_path = 'F1dataset' 
database_path = 'f1_project.db'

con = sqlite3.connect(database_path)

csv_files = glob.glob(os.path.join(csv_folder_path, '*.csv'))

for file in csv_files:
    df = pd.read_csv(file)
    
    table_name = os.path.splitext(os.path.basename(file))[0]
    
    df.to_sql(table_name, con, if_exists='replace', index=False)
    
    print(f"Loaded {table_name} into the database.")

Loaded circuits into the database.
Loaded constructors into the database.
Loaded constructor_results into the database.
Loaded constructor_standings into the database.
Loaded drivers into the database.
Loaded driver_standings into the database.
Loaded lap_times into the database.
Loaded pit_stops into the database.
Loaded qualifying into the database.
Loaded races into the database.
Loaded results into the database.
Loaded seasons into the database.
Loaded sprint_results into the database.
Loaded status into the database.


# Load data from Kaggle CSV files

In [ ]:
analysis_query = """
                SELECT *
                FROM races AS rc
                INNER JOIN results AS re ON rc.raceID = re.raceID
                INNER JOIN drivers AS dv ON re.driverID = dv.driverID
                INNER JOIN constructors AS cs ON re.constructorId = cs.constructorId
                INNER JOIN qualifying AS qf ON rc.raceID = qf.raceID AND re.driverID = qf.driverID
                INNER JOIN status AS st on re.statusID = st.statusID
                WHERE year >= 2021
                AND year <= 2024
                AND status = 'Finished'
                """


con = sqlite3.connect("f1_project.db")

analysis_df = pd.read_sql_query(analysis_query, con)

analysis_df.head()

,raceId,year,round,circuitId,name,date,time,url,fp1_date,fp1_time,...,raceId,driverId,constructorId,number,position,q1,q2,q3,statusId,status
0,1052,2021,1,3,Bahrain Grand Prix,2021-03-28,15:00:00,http://en.wikipedia.org/wiki/2021_Bahrain_Gran...,2021-03-26,\N,...,1052,1,131,44,2,1:30.617,1:30.085,1:29.385,1,Finished
1,1052,2021,1,3,Bahrain Grand Prix,2021-03-28,15:00:00,http://en.wikipedia.org/wiki/2021_Bahrain_Gran...,2021-03-26,\N,...,1052,830,9,33,1,1:30.499,1:30.318,1:28.997,1,Finished
2,1052,2021,1,3,Bahrain Grand Prix,2021-03-28,15:00:00,http://en.wikipedia.org/wiki/2021_Bahrain_Gran...,2021-03-26,\N,...,1052,822,131,77,3,1:31.200,1:30.186,1:29.586,1,Finished
3,1052,2021,1,3,Bahrain Grand Prix,2021-03-28,15:00:00,http://en.wikipedia.org/wiki/2021_Bahrain_Gran...,2021-03-26,\N,...,1052,846,1,4,7,1:30.902,1:30.099,1:29.974,1,Finished
4,1052,2021,1,3,Bahrain Grand Prix,2021-03-28,15:00:00,http://en.wikipedia.org/wiki/2021_Bahrain_Gran...,2021-03-26,\N,...,1052,815,9,11,11,1:31.165,1:30.659,\N,1,Finished


# Merge the dataframes

In [ ]:
df = pd.merge(races_df, results_df, on='raceId')
df = pd.merge(df, drivers_df, on='driverId')
df = pd.merge(df, constructors_df, on='constructorId')
df = pd.merge(df, qualifying_df, on=['raceId', 'driverId'])
df = pd.merge(df, status_df, on='statusId')


df['driver'] = df['forename'] + ' ' + df['surname']

# Clean and prepare data

In [ ]:
analysis_df = df[df['year'] >= 2021].copy()
analysis_df.rename(columns = {
    'name_x': 'raceName',
    'name_y': 'constructorName',
    'position_x': 'racePosition',
    'position_y': 'qualifyingPosition',
}, inplace=True)

columns_to_keep = [
    'year',
    'raceName',
    'driver',
    'constructorName',
    'grid',
    'qualifyingPosition', 
    'racePosition',
    'points',
    'laps',
    'status'
]

analysis_df = analysis_df[columns_to_keep]

# Filter to only include drivers who finished the race.
finished_statuses = ['Finished', '+1 Lap', '+2 Laps', '+3 Laps', '+4 Laps']
analysis_finished_df = analysis_df[analysis_df['status'].isin(finished_statuses)].copy()

analysis_finished_df['racePosition'] = pd.to_numeric(analysis_finished_df['racePosition'], errors='coerce')
analysis_finished_df['grid'] = pd.to_numeric(analysis_finished_df['grid'], errors='coerce')


# Creating visualizations

In [ ]:
plt.figure(figsize=(10, 6))
sns.regplot(data=analysis_finished_df, x='grid', y='racePosition')
plt.title('Qualifying Position vs. Race Finish Position (2021-2024)')
plt.xlabel('Qualifying Position (Grid)')
plt.ylabel('Race Position (racePosition)')

plt.show()

The regression plot above visualizes the relationship between a driver's starting grid position and their final race position for the 2021-2024 F1 seasons.

The clear upward-sloping trendline shows a strong positive correlation between the two variables. This indicates that, on average, a driver's starting position is a strong predictor of their finishing position.

In [ ]:
analysis_finished_df['position_change'] = analysis_finished_df['grid'] - analysis_finished_df['racePosition']
median_change = analysis_finished_df['position_change'].median()

plt.figure(figsize=(12, 6))
sns.histplot(analysis_finished_df, x = 'position_change', binwidth = 1, kde=True)
plt.axvline(0, color='red', linestyle='--', label='No position change')
plt.axvline(median_change, color='green', linestyle='--', label=f'Median of position change: {median_change:.2f}')
plt.legend()

plt.title('Distribution of Race Finish Positions Changes (2021-2024)')
plt.xlabel('Position Gain or Lost (grid - racePosition)')
plt.ylabel('Count of Occurrences')

plt.show()

The histogram central peak is at the range [0,1], indicating that most common outcome is drivers finishing their race in or very near their starting grid position.

Also, from the distribution, we can see that large position changes (gain or lose position over 10), are rarely happened.

The median of the distribution is 1.0. A positive median suggests that drivers are more likely to gain position rather than lose position.

In [ ]:
driver_performance = analysis_finished_df.groupby('driver')['position_change'].mean().sort_values(ascending=False).reset_index()

plt.figure(figsize=(12, 10))
sns.barplot(data=driver_performance, x='position_change', y='driver', orient='h')

plt.title('Average Position Change by Driver (2021-2024)')
plt.xlabel('Average Position Change (grid - racePosition)')
plt.ylabel('Driver')

plt.show()

In [ ]:
accident_statuses = ['Collision', 'Accident', 'Spun off']
accident_incidents = analysis_df[analysis_df['status'].isin(accident_statuses)]
accident_counts = accident_incidents['driver'].value_counts().reset_index()
accident_counts.columns = ['driver', 'numberOfAccidents']

plt.figure(figsize=(12, 10))
sns.barplot(data=accident_counts, x='numberOfAccidents', y='driver', orient='h')

plt.title('Drivers with Most Incidents (Collision, Accident, Spun Off) in 2021-2024')
plt.xlabel('Number of Incidents')
plt.ylabel('Driver')

plt.show()

In [ ]:
circuit_get_most_accidents = accident_incidents['raceName'].value_counts().reset_index()
circuit_get_most_accidents.columns = ['raceName', 'numberOfAccidents']
circuit_get_most_accidents = circuit_get_most_accidents.sort_values(by='numberOfAccidents', ascending=False)

plt.figure(figsize=(12, 10))
sns.barplot(data=circuit_get_most_accidents, x='numberOfAccidents', y='raceName', orient='h')

plt.title('Circuits with Most Incidents (Collision, Accident, Spun Off) in 2021-2024')
plt.xlabel('Number of Incidents')
plt.ylabel('Circuit (raceName)')

plt.show()

In [ ]:
circuit_get_most_positions = analysis_finished_df.groupby('raceName')['position_change'].mean()
circuit_get_most_positions = circuit_get_most_positions.sort_values(ascending=False)
circuit_get_most_positions = circuit_get_most_positions.reset_index()

plt.figure(figsize=(12, 10))
sns.barplot(data=circuit_get_most_positions, x='position_change', y='raceName', orient='h')

plt.title('Average Position Change by Circuit (2021-2024)')
plt.xlabel('Average Position Change (grid - racePosition)')
plt.ylabel('Circuit Name')

plt.show()

Next, I want to investgate "why", and answer the questions below:

1. Circuit Type: Does the type of circuit (e.g., a high-speed circuit like Monza vs. a low-speed circuit like Monaco) have a significant impact on overall position changes?

2. Race Chaos: Do races with more incidents (collisions, accidents, spins) show a wider distribution of position changes?

In [ ]:
circuit_map = {'Australian Grand Prix': 'High Speed',
    'Italian Grand Prix': 'High Speed',
    'Styrian Grand Prix': 'High Speed',
    'Bahrain Grand Prix': 'Medium Speed',
    'Japanese Grand Prix': 'High Speed',
    'Abu Dhabi Grand Prix': 'Low Speed',
    'Chinese Grand Prix': 'Medium Speed',
    'Saudi Arabian Grand Prix': 'High Speed',
    'Monaco Grand Prix': 'Low Speed',
    'São Paulo Grand Prix': 'Low Speed',
    'British Grand Prix': 'High Speed',
    'Emilia Romagna Grand Prix': 'Medium Speed',
    'Hungarian Grand Prix': 'Low Speed',
    'Mexico City Grand Prix': 'Low Speed',
    'Qatar Grand Prix': 'Medium Speed',
    'United States Grand Prix': 'Medium Speed',
    'Singapore Grand Prix': 'Low Speed',
    'Azerbaijan Grand Prix': 'Medium Speed',
    'Canadian Grand Prix': 'Medium Speed',
    'Miami Grand Prix': 'Medium Speed',
    'Portuguese Grand Prix': 'Medium Speed',
    'Las Vegas Grand Prix': 'Medium Speed',
    'French Grand Prix': 'Medium Speed',
    'Austrian Grand Prix': 'High Speed',
    'Spanish Grand Prix': 'Medium Speed',
    'Turkish Grand Prix': 'Medium Speed',
    'Belgian Grand Prix': 'High Speed',
    'Dutch Grand Prix': 'Low Speed'
}

analysis_finished_df['circuitType'] = analysis_finished_df['raceName'].map(circuit_map)
circuit_type_performance = analysis_finished_df.groupby('circuitType')['position_change'].mean().sort_values(ascending=False).reset_index()

plt.figure(figsize=(8, 6))
sns.barplot(data=circuit_type_performance, x='position_change', y='circuitType', orient='h')

plt.title('Average Position Change by Circuit Type (2021-2024)')
plt.xlabel('Average Position Change (grid - racePosition)')
plt.ylabel('Circuit Type')

plt.show()

High Speed circuits yield the highest position gain(~0.75);  Low Speed circuits also show high gain(~0.6); Medium Speed circuits show the lowest gain (~0.49).